In [3]:
import pandas as pd
import numpy as np

In [25]:
FILEPATH = "Y:/Members_Current/Jenn/EEG study/Imported data/cleaned/"
participants = ["105", "106", "107"] #, "904", "905", "906"]

dfs = [pd.read_csv("%s%s_cleaned_ml.csv" % (FILEPATH, s)) for s in participants]

df = pd.concat(dfs, axis=0, ignore_index=True)

In [27]:
# samples are df rows mod 1200, pre is 0-199, post is 200-1199

# extract pre rows and post rows, randomly choose 200-400ms as post
pre = df[df.Time < 0]
post = df[df.Time >= 200][df.Time < 400]

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [28]:
pre = pre.drop(columns=["Time", "E65"], axis=1)
post = post.drop(columns=["Time", "E65"], axis=1)

In [29]:
timeFrame = 200

preX = pre.values
(i,j) = preX.shape
preX = np.reshape(preX, (i // 200, j * 200))

In [30]:
postX = post.values
(i,j) = postX.shape
postX = np.reshape(postX, (i // 200, j * 200))

In [31]:
(preN, preD) = preX.shape
(postN, postD) = postX.shape

In [32]:
X = np.concatenate((preX, postX), axis=0)
(n, d) = X.shape
assert(n == preN + postN)
assert(d == postD)
assert(d == preD)

In [33]:
y = np.concatenate((np.zeros(preN), np.ones(postN)))
assert(y.shape == (n,))

In [34]:
rand_indices = np.random.permutation(n)
split = int(n * 0.8)
train_idx, valid_idx = rand_indices[:split], rand_indices[split:]
train_X, valid_X, train_y, valid_y = X[train_idx, :], X[valid_idx, :], y[train_idx], y[valid_idx]

In [35]:
# from sklearn.svm import SVC
# model = SVC(gamma='auto', kernel='linear')
# model.fit(train_X, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [47]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_X, train_y)

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [48]:
Xhat = model.predict(valid_X)

In [49]:
np.mean(Xhat != valid_y)

0.09333333333333334

In [50]:
Xhat

array([1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 1., 0., 0., 1., 1.])

In [51]:
valid_y

array([1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1.,
       1., 0., 1., 0., 0., 1., 1.])

In [52]:
np.mean(model.predict(train_X) != train_y)

0.0033444816053511705

In [53]:
Xhat != valid_y

array([False, False, False, False, False,  True,  True, False, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
       False,  True, False, False, False, False, False, False, False,
       False, False, False])